In [1]:
import subprocess
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = 'Arial'
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

# read in the data

In [24]:
os.chdir("/mnt/Storage/home/wangyiman/CHMsInOtherContexts/CellStateTransition/CHMOrganization/Universal_specific/Features/ChromatinStates/Cavalli_Cell2017_ABcompartment_5tissue/CHM_intersect_table")
count_dict = {
    'ESCs -> NPCs' : {},
    'NPCs -> CNs' : {},
    'CNs -> ncxNPCs' : {},
    'ncxNPCs -> ncxCNs' : {},
    # 'ncxCNs' : {}
}
for process in ['Universal', 
                # 'Universal_complementarySet', 
                # 'EarlyEmbryogenesis_complementarySet', 'EarlyEmbryogenesis_complementarySet_otherCommon',
                # 'PGCsDevelopment_complementarySet', 'PGCsDevelopment_complementarySet_otherCommon',
                # 'Spermatogenesis_complementarySet', 'Spermatogenesis_complementarySet_otherCommon',
                # 'RetinalDevelopment_complementarySet', 'RetinalDevelopment_complementarySet_otherCommon',
                # 'HeartDevelopment_complementarySet', 'HeartDevelopment_complementarySet_otherCommon',
                # 'LiverDevelopment_complementarySet', 'LiverDevelopment_complementarySet_otherCommon',
                'EarlyEmbryoSpecific', 
                'PGCSpecific', 
                'SpermSpecific', 
                'RetinalSpecific', 
                'HeartSpecific', 
                'LiverSpecific',
                'NonCHMsCpGrich', 
                ] :
    intsct_df_binary = pd.DataFrame()
    ### A : 4, B : 2, other : 1
    process_l = f'{process}CHM' if process != 'NonCHMsCpGrich' else process
    for i,compart in enumerate(['A', 'B', 'others']) :
        df = pd.read_csv(
            f'{process_l}_5tissue_{compart}.bed',
            sep = "\t",
            header = None,
            usecols = [3,4,5,6,7]
                        )
        if i == 0 :
            intsct_df_binary = df * (2 ** (2-i))
        else :
            intsct_df_binary = intsct_df_binary + df * (2 ** (2-i))
            
            
    intsct_df_binary = intsct_df_binary.loc[[(len(set(intsct_df_binary.iloc[i,:])) > 1) for i in range(intsct_df_binary.shape[0])]]
    
    col_label_next = {
        'ESCs' : 'NPCs',
        'NPCs' : 'CNs',
        'CNs' : 'ncxNPCs',
        'ncxNPCs' : 'ncxCNs',
}
    compart_class_value = {
        'A': [4, 5, 6, 7],
        'B': [2, 3, 6, 7],
        'others': [1]
    }
            
    for i,key in enumerate(['ESCs -> NPCs', 'NPCs -> CNs', 'CNs -> ncxNPCs', 'ncxNPCs -> ncxCNs']) :
        col = i + 3
        for compart_class_1 in ['A', 'B', 'others'] :
            for compart_class_2 in ['A', 'B', 'others'] :
                count_dict[key][f'{process}: {compart_class_1} -> {compart_class_2}'] = (intsct_df_binary[col].isin(compart_class_value[compart_class_1]) & intsct_df_binary[col+1].isin(compart_class_value[compart_class_2])).sum()
        
count_df = pd.DataFrame(count_dict)

In [25]:
intsct_df_binary

,3,4,5,6,7
3,4,2,2,2,2
4,4,2,2,2,2
5,4,2,2,2,2
6,4,2,2,2,2
7,4,2,2,2,2
...,...,...,...,...,...
22035,4,1,4,1,4
22057,1,4,4,4,4
22058,1,4,4,4,4
22063,4,4,4,4,1


In [26]:
count_df

,ESCs -> NPCs,NPCs -> CNs,CNs -> ncxNPCs,ncxNPCs -> ncxCNs
Universal: A -> A,135,491,278,180
Universal: A -> B,21,93,306,116
Universal: A -> others,19,27,31,20
Universal: B -> A,435,97,21,46
Universal: B -> B,154,93,133,400
...,...,...,...,...
NonCHMsCpGrich: B -> B,202,190,431,995
NonCHMsCpGrich: B -> others,202,30,61,175
NonCHMsCpGrich: others -> A,1213,619,567,594
NonCHMsCpGrich: others -> B,40,38,116,98


In [27]:
data = count_df.to_dict()

row_labels = count_df.reset_index()['index'].str.split(' -', expand=True)[0].unique().tolist()
column_labels = count_df.T.reset_index()['index'].str.split(' -', expand=True)[0].tolist() + [count_df.T.reset_index()['index'].str.split('> ', expand=True)[1].tolist()[-1]]

label = [f'{x} in {y}' for x in row_labels for y in column_labels]

In [32]:
# Initialize source, target, and value lists
source = []
target = []
value = []

col_label_next = {
    'ESCs' : 'NPCs',
    'NPCs' : 'CNs',
    'CNs' : 'ncxNPCs',
    'ncxNPCs' : 'ncxCNs',
}

# Populate the source, target, and value lists
for col_index, col_label in enumerate(column_labels[:-1]):
    for process in ['Universal', 
                # 'Universal_complementarySet', 
                'EarlyEmbryoSpecific', 
                'PGCSpecific', 
                'SpermSpecific', 
                'RetinalSpecific', 
                'HeartSpecific', 
                'LiverSpecific',
                # 'NonCHMsCpGrich', 
                ] :
        for class_1 in ['A', 'B', 'others'] :
            for class_2 in ['A', 'B', 'others'] :
                data_key = f'{process}: {class_1} -> {class_2}'
                # print(data_key)
                source.append(label.index(f'{process}: {class_1} in {col_label}'))
                target.append(label.index(f'{process}: {class_2} in {col_label_next[col_label]}'))
                value.append(data[f'{col_label} -> {col_label_next[col_label]}'][f'{process}: {class_1} -> {class_2}'])


# Print the resulting lists
# print("Source:", source)
# print("Target:", target)
# print("Label:", label)
# print("Value:", value)

In [33]:
len(value)

252

In [34]:
3 * 8 * 4

96

In [35]:
import plotly.graph_objects as go
import plotly.io as pio

flow_colors = ['#C490C6', '#90ADC6', '#777777'] * 3 * 8 * 4

# Create the alluvial diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        # label=label
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=flow_colors
    )
)])

# Update the layout and show the plot
fig.update_layout(
    title_text="",
    font_size=10,
    width=1000,  # Custom figure width
    height=1000  # Custom figure height
)
fig.show()

# Save the figure as a PDF
pio.write_image(fig, '/mnt/Storage/home/wangyiman/CHMsInOtherContexts/figures/make13_alluvial_inconsist_universalProcessSpecificNonCHM_ABOthers.pdf')


In [36]:
import plotly.graph_objects as go

# Define the source, target, value, and label data
# source = [0, 0, 1, 1, 1, 6, 6, 7, 7, 8, 8]
# target = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# value = [8, 4, 2, 8, 4, 2, 8, 4, 6, 4, 3]
# label = ["A", "B", "C1", "C2", "C3", "D1", "D2", "E1", "E2", "F1", "F2", "G1", "G2"]

flow_colors = ['#C490C6', '#90ADC6', '#777777'] * 3 * 8 * 4

# Create the alluvial diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=label
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        color=flow_colors
    )
)])

# Update the layout and show the plot
fig.update_layout(
    title_text="Alluvial Diagram Example",
    font_size=10,
    width=1000,  # Custom figure width
    height=1000  # Custom figure height
)
fig.show()
